# Read pickle files 

In [83]:
import pickle
import re 
import numpy as np

In [26]:

# file_path = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/'#/resxVecFold0/'
#file_path='/home/sjoshi/codes/python/BeatPD/code/'
# file_name = 'objs_50_k_6.pkl'


[glob_trai_pred,glob_trai_true, glob_test_pred,glob_test_true, \
 mse_training_per_subjectid, mse_test_per_subjectid, \
 train_nb_files_per_subjectid, test_nb_files_per_subjectid] = pickle.load( open( file_path+file_name, "rb" ) )

FileNotFoundError: [Errno 2] No such file or directory: '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/objs_50_k_6.pkl'

In [59]:
import glob

lResxFolders = [f for f in glob.glob(file_path + "resx*")]
sPatternFold = '(?<=[Ff]old)\d+'
print(lResxFolders)

lFolds = []
for folder in lResxFolders:
    lFolds.append(re.findall(sPatternFold, folder)[0])
print(lFolds)


['/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/resxVecFold0', '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/resxVecFold1', '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/resxVecFold2', '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/resxVecFold3', '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/resxVecFold4']
['0', '1', '2', '3', '4']
['50']


#### With (or without?) knn

In [110]:
def read_pickle(sFilePath, bKnn):
    lResxFolders = [f for f in glob.glob(sFilePath + "resx*")]
    sPatternFold = '(?<=[Ff]old)\d+'
    lComponents = [] 
    lNeighbors = [] 

    # Building the list of folders we have to open 
    lResxFolders = [f for f in glob.glob(sFilePath + "resx*")]

    # Get a list of all files starting with objs
    for fold_folder in lResxFolders:
        lObjsFiles = glob.glob(fold_folder + "/objs*")

    # Get a list of all no of components 
    for objsFile in lObjsFiles:
        # Get a list of all iComponents
        sPatternComponents = r'\d+(?=[_|.])'
        lComponents.append(re.findall(sPatternComponents, objsFile)[0])

        # Get a list of all iNeighbors
        sPatternNeighbors = r'(?<=k_)\d+'
        lNeighbors.append(re.findall(sPatternNeighbors, objsFile)[0])

    print('Components found : ', lComponents)
    print('Neighbors found : ', lNeighbors)
    
    for neighbor in lNeighbors:
        print('------ FOR NEIGHBOR ', neighbor, '------')
        for component in lComponents:
            print('---- FOR COMPONENT ', component, '----')
            
            # To compute mean accuracy accross all folds
            allfolds_glob_trai_pred = []
            allfolds_glob_trai_true = []
            allfolds_glob_test_pred = []
            allfolds_glob_test_true = [] 
            
            # To compute final score across all folds 
            allfolds_mse_training_per_subjectid = []
            allfolds_mse_test_per_subjectid = []
            allfolds_train_nb_files_per_subjectid = []
            allfolds_test_nb_files_per_subjectid = []
            
            for fold_folder in lResxFolders:
                print(fold_folder)

                if bKnn:
                    sFileName = fold_folder+'/objs_'+component+'_k_'+neighbor+'.pkl'
                else:
                    sFileName = fold_folder+'/objs_'+component+'.pkl'

                # Retrieve DataFrames from Pickle file 
                [glob_trai_pred,glob_trai_true, \
                 glob_test_pred,glob_test_true, \
                 mse_training_per_subjectid, \
                 mse_test_per_subjectid, \
                 train_nb_files_per_subjectid, \
                 test_nb_files_per_subjectid] = pickle.load(open(sFileName, "rb" ) )

                # Build the DataFrames for all folds 
                allfolds_glob_trai_pred = np.append(allfolds_glob_trai_pred, glob_trai_pred)
                allfolds_glob_trai_true = np.append(allfolds_glob_trai_true, glob_trai_true)
                allfolds_glob_test_pred = np.append(allfolds_glob_test_pred, glob_test_pred)
                allfolds_glob_test_true = np.append(allfolds_glob_test_true, glob_test_true)

                allfolds_mse_training_per_subjectid = np.append(allfolds_mse_training_per_subjectid, 
                                                                mse_training_per_subjectid)
                allfolds_mse_test_per_subjectid = np.append(allfolds_mse_test_per_subjectid,
                                                           mse_test_per_subjectid)
                allfolds_train_nb_files_per_subjectid = np.append(allfolds_train_nb_files_per_subjectid, 
                                                                 train_nb_files_per_subjectid)
                allfolds_test_nb_files_per_subjectid = np.append(allfolds_test_nb_files_per_subjectid, 
                                                                test_nb_files_per_subjectid)
            
            print('Global training accuracy: {}'.format((allfolds_glob_trai_true == allfolds_glob_trai_pred).mean()))
            print('Global testing accuracy: {}'.format((allfolds_glob_test_true == allfolds_glob_test_pred).mean()))
            
            final_score(allfolds_mse_training_per_subjectid,
                        allfolds_train_nb_files_per_subjectid,
                        training_or_test='Train ')
            final_score(allfolds_mse_test_per_subjectid,
                        allfolds_test_nb_files_per_subjectid,
                        training_or_test='Test ')
            

#file_path = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/'
sFilePath = '/home/sjoshi/codes/python/BeatPD/code/'
bKnn = True # Should we use KNN neighbors or not?
read_pickle(sFilePath, bKnn)

Components found :  ['50']
Neighbors found :  ['3']
------ FOR NEIGHBOR  3 ------
---- FOR COMPONENT  50 ----
/home/sjoshi/codes/python/BeatPD/code/resxVecFold1
/home/sjoshi/codes/python/BeatPD/code/resxVecFold4
/home/sjoshi/codes/python/BeatPD/code/resxVecFold0
/home/sjoshi/codes/python/BeatPD/code/resxVecFold3
/home/sjoshi/codes/python/BeatPD/code/resxVecFold2
Global training accuracy: 0.6154401154401155
Global testing accuracy: 0.5380577427821522
Train Final score :  0.01728909175904204
Test Final score :  0.10001114592895653


#### Should work if not knn

In [100]:
#file_path = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/'
file_path = '/home/sjoshi/codes/python/BeatPD/code/'

lResxFolders = [f for f in glob.glob(file_path + "resx*")]
sPatternFold = '(?<=[Ff]old)\d+'
lComponents = [] 
lNeighbors = [] 

# Building the list of folders we have to open 
lResxFolders = [f for f in glob.glob(file_path + "resx*")]

# Get a list of all files starting with objs
for fold_folder in lResxFolders:
    lObjsFiles = glob.glob(fold_folder + "/objs*")

# Get a list of all no of components 
for objsFile in lObjsFiles:
    # Get a list of all iComponents
    sPatternComponents = r'\d+(?=[_|.])'
    lComponents.append(re.findall(sPatternComponents, objsFile)[0])
    
    # Get a list of all iNeighbors
#     sPatternNeighbors = r'(?<=k_)\d+'
#     lNeighbors.append(re.findall(sPatternNeighbors, objsFile)[0])

print(lComponents)

for component in lComponents:
    allfolds_glob_trai_pred = []
    allfolds_glob_trai_true = []
    allfolds_glob_test_pred = []
    allfolds_glob_test_true = [] 

    print('------ FOR COMPONENT ', component, '------')
    for fold_folder in lResxFolders:
        print(fold_folder)
        file_name = fold_folder+'/objs_'+component+'.pkl'
        [glob_trai_pred,glob_trai_true, \
         glob_test_pred,glob_test_true, \
         ] = pickle.load(open(file_name, "rb" ))
        
        allfolds_glob_trai_pred = np.append(allfolds_glob_trai_pred, glob_trai_pred)
        allfolds_glob_trai_true = np.append(allfolds_glob_trai_true, glob_trai_true)
        allfolds_glob_test_pred = np.append(allfolds_glob_test_pred, glob_test_pred)
        allfolds_glob_test_true = np.append(allfolds_glob_test_true, glob_test_true)
#         [glob_trai_pred,glob_trai_true, \
#          glob_test_pred,glob_test_true, \
#          mse_training_per_subjectid, \
#          mse_test_per_subjectid, \
#          train_nb_files_per_subjectid, \
#          test_nb_files_per_subjectid] = pickle.load(open(file_name, "rb" ) )


    print('Global training accuracy: {}'.format((allfolds_glob_trai_true == allfolds_glob_trai_pred).mean()))
    print('Global testing accuracy: {}'.format((allfolds_glob_test_true == allfolds_glob_test_pred).mean()))

['50']
------ FOR COMPONENT  50 ------
/home/sjoshi/codes/python/BeatPD/code/resxVecFold1


FileNotFoundError: [Errno 2] No such file or directory: '/home/sjoshi/codes/python/BeatPD/code/resxVecFold1/objs_50.pkl'

In [58]:
# From exp folder, find "resx*" folders 
file_path = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/'

# Read all Fold0 to 4 
    # Look for obs files


# Get a list of all iComponents
sPatternComponents = r'\d+(?=[_])'
lComponents = re.findall(sPatternComponents, file_name)

# Get a list of all iNeighbors
sPatternNeighbors = r'(?<=k_)\d+'
lNeighbors = re.findall(sPatternNeighbors, file_name)



# List files in given folder 
path = file_path
files = [i for i in os.listdir(path)if os.path.isfile(os.path.join(path,i))]
print(files)

[]


In [104]:
from sklearn.metrics import mean_squared_error
from math import sqrt 

def final_score(mse_per_subjectid, nb_files_per_subject_id, training_or_test=''):
    numerator = np.sum([a * b for a, b in zip(np.sqrt(nb_files_per_subject_id), mse_per_subjectid)])
    denominator = np.sum(np.sqrt(nb_files_per_subject_id))
#     print('numerator : ', numerator)
#     print('denominator : ', denominator)
    print(training_or_test+'Final score : ', np.divide(numerator, denominator))